In [1]:
!pip install transformers datasets evaluate sacrebleu transformers[torch] sentencepiece

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/0a/a6/2ac47e71e526bbcd97ea08f20d9ef7d3852e2594ec7b2d55f5d2bbfd7aae/sacrebleu-2.3.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.4 MB/s eta 0:00:00
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 12.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer, MarianConfig, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,  PreTrainedTokenizerFast, AutoTokenizer
#from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
import torch
import evaluate
import numpy as np
import json

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
device = "cuda:0"

In [14]:
HEADS = 4
ENCODER_LAYERS = 6
DECODER_LAYERS = 6
HIDDEN_LAYERS = 6
VOCAB_SIZE = 15000
EPOCHS = 32
BATCH_SIZE = 64
INIT_LR = 1e-4
BASE_MODEL = "Helsinki-NLP/opus-mt-es-fi"
TRANSFER_LEARNING = True # Keep all other values consistent with the configurations of the model
USE_DICT = True
SOURCE_LANG = 'pbb'
TARGET_LANG = 'spa'

In [5]:
books = load_dataset("Gaxys/nasa_spa")

books = books["train"].train_test_split(test_size=0.2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/Gaxys--nasa_spa-57b1f0496f1d27f0/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
if USE_DICT:
    dictionary = load_dataset("Gaxys/nasa_spa_dict")
    dictionary = dictionary["train"].train_test_split(test_size=0.2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/Gaxys--nasa_spa_dict-551352ed875a0412/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, model_max_length=128)

In [8]:
tokenizer = tokenizer.train_new_from_iterator(books['train'], VOCAB_SIZE)

In [9]:
pre_trained_config = {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": False,
  "add_final_layer_norm": False,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      VOCAB_SIZE - 1
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8 if TRANSFER_LEARNING else HEADS,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6 if TRANSFER_LEARNING else DECODER_LAYERS,
  "decoder_start_token_id": VOCAB_SIZE - 1,
  "decoder_vocab_size": VOCAB_SIZE,
  "dropout": 0.1,
  "encoder_attention_heads": 8 if TRANSFER_LEARNING else HEADS,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6 if TRANSFER_LEARNING else ENCODER_LAYERS,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": True,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_length": 512,
  "max_position_embeddings": 512,
  "model_type": "marian",
  "normalize_before": False,
  "normalize_embedding": False,
  "num_beams": 4,
  "num_hidden_layers": 6 if TRANSFER_LEARNING else HIDDEN_LAYERS,
  "pad_token_id": VOCAB_SIZE - 1,
  "scale_embedding": True,
  "share_encoder_decoder_embeddings": True,
  "static_position_embeddings": True,
  "transformers_version": "4.35.2",
  "use_cache": True,
  "vocab_size": VOCAB_SIZE
}

In [10]:
configuration = MarianConfig(**pre_trained_config)

In [11]:
model = MarianMTModel(configuration)

In [12]:
tuned_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-es-fi")

In [13]:
parts = list(dict(model.named_parameters()).keys())
parts.remove('model.shared.weight')

def transfer (tuned, to_tune, parts):
    target = dict(to_tune.named_parameters())
    source = dict(tuned.named_parameters())

    for part in parts:
        target[part].data.copy_(source[part].data)

if TRANSFER_LEARNING:
    transfer(tuned_model, model, parts)

In [16]:
source_lang = SOURCE_LANG
target_lang = TARGET_LANG


def preprocess_function(examples):
    inputs = [example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [17]:
if USE_DICT:
    tokenized_dict = dictionary.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
tokenized_books = books.map(preprocess_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [20]:
metric = evaluate.load("sacrebleu")

In [21]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [22]:
training_args = Seq2SeqTrainingArguments(
    output_dir="transfer_with_dict_nasa_15k_1e-4",
    evaluation_strategy="epoch",
    learning_rate=INIT_LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    #fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True,
    save_strategy="epoch"
)

#fast_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dict["train"],
    eval_dataset=tokenized_dict["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

#trainer.train()

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir="transfer_no_dict_nasa_15k_1e-4",
    evaluation_strategy="epoch",
    learning_rate=INIT_LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True,
    save_strategy="epoch"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
    #compute_metrics=compute_metrics,
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,2.084489
2,No log,1.618965
3,No log,1.487017
4,No log,1.405701
5,1.843900,1.339887
6,1.843900,1.300080
7,1.843900,1.274181
8,1.843900,1.244222
9,1.843900,1.228137
10,1.303400,1.207933


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


TrainOutput(global_step=3328, training_loss=1.2263239713815541, metrics={'train_runtime': 2014.1119, 'train_samples_per_second': 105.734, 'train_steps_per_second': 1.652, 'total_flos': 7218995795066880.0, 'train_loss': 1.2263239713815541, 'epoch': 32.0})

In [25]:
eval_preds = trainer.predict(tokenized_books['test'])

In [26]:
result = compute_metrics((eval_preds.predictions, eval_preds.label_ids))

In [27]:
result

{'bleu': 1.3122, 'gen_len': 511.0}

In [ ]:
with open("non_transfer_4_head_6_layer_with_dict_nasa_15k_1e-4.json", 'w') as f:
    json.dump(trainer.state.log_history, f)

In [ ]:
with open("non_transfer_4_head_6_layer_with_dict_nasa_15k_1e-4_result.json", 'w') as f:
    json.dump(result, f)